## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import sklearn
assert sklearn.__version__ >= "0.20"

# 시드 고정
np.random.seed(21)

## 데이터

### 데이터 가져오기

In [2]:
origin_data = pd.read_excel('./3차_VOD_2308.xlsx', index_col = None)

### 데이터 전처리

In [3]:
# 불필요한 컬럼 제거
drop_data = origin_data.drop(['Unnamed: 0', 'ct_cl', 'genre_of_ct_cl', 'use_tms', 'SMRY', 'disp_rtm', 'strt_dt'], axis = 1)

In [4]:
# 데이터에서 불필요한 부분 제거

import re

# 제목 데이터에서 불필요한 부분을 제거하고
# 전처리된 제목을 새로운 컬럼으로 추가하는 함수
def get_rid_asset(dataframe):
    # 괄호 안의 불필요한 부분 패턴
    pattern = re.compile('\([A-Za-z가-힣\s]+\)|\([가-힣\d\s]+\)|\([\d\/]{5,}\)')

    new_data = []

    for item in dataframe['asset_nm']:
        name = str(item)
        match_pattern = pattern.findall(name)

        for dt in match_pattern:
            name = name.replace(dt, "")
            #new_data.append(dt)

        new_data.append(name) 
    
    # 제목과 함께 존재하는 회차 정보 제거
    pattern = re.compile('\s[\d]+회')

    series = []

    for item in new_data:
        name = str(item)

        # 정규 표현식을 만족하는 경우 - 드라마 컨텐츠
        try:
            match_pattern = pattern.findall(name)

            for dt in match_pattern:
                name = name.replace(dt, "")
            series.append(name)

        # 정규 표현식을 만족하지 않는 경우 - 영화(~~회 가 없음)
        except:
            series.append(item)
    
    # 데이터 프레임에 처리 결과 컬럼을 추가
    dataframe['series_name'] = series

In [5]:
# 데이터에 함수를 적용
get_rid_asset(drop_data)

# 함수 적용 결과 확인
#print(drop_data.head())

In [6]:
# 출연진 정보가 없는 데이터 제거
no_actr_data = drop_data.drop(drop_data[drop_data['ACTR_DISP'] == '-'].index, axis = 0)

# 제거 확인 - 출력되는 데이터 없음
# print(no_actr_data[no_actr_data['ACTR_DISP'] == '-'])

In [7]:
drop_data.head()

,subsr,asset_nm,ACTR_DISP,series_name
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),"장도연,장현성,장성규",꼬리에꼬리를무는그날이야기


In [48]:
# 특정 subsr 의 시청 데이터에서 출연진 가져오기
view_data = no_actr_data[drop_data['subsr'] == 64385000]

# 데이터 확인
print(view_data)
print(view_data.groupby('series_name').count())

         subsr                     asset_nm        ACTR_DISP    series_name
45    64385000  꼬리에꼬리를무는그날이야기 40회(22/08/11)      장도연,장현성,장성규  꼬리에꼬리를무는그날이야기
60    64385000    과학수사대 스모킹 건 05회(23/04/26)      안현모,이혜원,유성호    과학수사대 스모킹 건
68    64385000    과학수사대 스모킹 건 03회(23/04/12)      안현모,이혜원,유성호    과학수사대 스모킹 건
125   64385000        동물극장 단짝 29회(22/08/27)          이금희,박명수        동물극장 단짝
126   64385000        동물극장 단짝 29회(22/08/27)          이금희,박명수        동물극장 단짝
...        ...                          ...              ...            ...
2173  64385000          웃는 사장 02회(23/07/02)       이경규,박나래,강율          웃는 사장
2209  64385000          웃는 사장 01회(23/06/25)       이경규,박나래,강율          웃는 사장
2210  64385000          웃는 사장 01회(23/06/25)       이경규,박나래,강율          웃는 사장
2291  64385000          닭, 싸움 01회(23/07/12)  장성규,이연복,여경옥,황제성          닭, 싸움
2292  64385000          닭, 싸움 01회(23/07/12)  장성규,이연복,여경옥,황제성          닭, 싸움

[373 rows x 4 columns]
                subsr  asset_nm  ACTR_DISP
series_name          

C:\Users\USER\AppData\Local\Temp\ipykernel_12024\643244924.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  view_data = no_actr_data[drop_data['subsr'] == 64385000]


## 출연진 기반 추천

### 출연진 데이터

In [36]:
# 출연진 정보를 저장할 list
actr_list = []

# 시청 데이터를 순회하면서 출연진 정보를 저장
for _, item in view_data.iterrows():
    if ',' in item['ACTR_DISP']:
        item_list = item['ACTR_DISP'].split(',')
    else:
        item_list = [item['ACTR_DISP']]
    
    for actr in item_list:
        actr_list.append(actr)

# 확인
#print(actr_list)

In [37]:
# counter 를 사용해서 출연 횟수 확인
from collections import Counter

count = Counter(actr_list)

# 가장 많이 출연한 상위 N 명의 출연진 정보
N_people = 7
bestN_actr = count.most_common(N_people)

In [38]:
# 상위 N위 출연진에 대해 가중치 설정
score_sum = 0
for actr_score in bestN_actr:
    score_sum += actr_score[1]

w_bestN = []
for actr_score in bestN_actr:
    w_bestN.append([actr_score[0], round((actr_score[1] / score_sum), 2)])
    
print(w_bestN)

[['신동엽', 0.17], ['정선희', 0.17], ['토니안 ', 0.17], ['조이', 0.17], ['이연복', 0.11], ['이정현', 0.11], ['이유리', 0.11]]


In [39]:
actr_score_list = []

# 각 행을 순회하면서 출연진 정보에 대한 점수를 더해서 데이터 생성
for _, item in drop_data.iterrows():
    score_data = 0
    
    for name_score in w_bestN:
        if name_score[0] in str(item['ACTR_DISP']):
            score_data += name_score[1]
    actr_score_list.append(score_data)
    
# 기존 데이터에 컬럼 추가
drop_data['ATCR_SCORE'] = actr_score_list

# 확인
drop_data.head()

,subsr,asset_nm,ACTR_DISP,series_name,ATCR_SCORE,rank
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다,0.0,37.0
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다,0.0,37.0
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다,0.0,37.0
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),김상중,그것이알고싶다,0.0,37.0
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),"장도연,장현성,장성규",꼬리에꼬리를무는그날이야기,0.0,242.0


In [40]:
# 출연진 점수를 기준으로 순위 부여
drop_data['rank'] = drop_data['ATCR_SCORE'].rank(method = 'min', na_option = 'bottom',
                                                    ascending = False)
# 확인
drop_data['ATCR_SCORE'].value_counts()

0.00    5318
0.68     118
0.17      79
0.33      61
0.11      39
Name: ATCR_SCORE, dtype: int64

In [41]:
drop_data[drop_data['ATCR_SCORE'] == 0.68]

,subsr,asset_nm,ACTR_DISP,series_name,ATCR_SCORE,rank
365,64385000,(HD)TV동물농장 1098회(22/12/11),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
366,64385000,(HD)TV동물농장 1098회(22/12/11),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
419,64385000,(HD)TV동물농장 1089회(22/10/02),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
420,64385000,(HD)TV동물농장 1089회(22/10/02),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
421,64385000,(HD)TV동물농장 1089회(22/10/02),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
...,...,...,...,...,...,...
1581,60918000,(HD)TV동물농장 1131회(23/07/30),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
1582,60918000,(HD)TV동물농장 1131회(23/07/30),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
1583,60918000,(HD)TV동물농장 1131회(23/07/30),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0
1584,64385000,(HD)TV동물농장 1095회(22/11/20),"신동엽,정선희,토니안 ,조이",TV동물농장,0.68,1.0


In [55]:
drop_data[drop_data['ATCR_SCORE'] == 0.33].groupby('series_name').count()

,subsr,asset_nm,ACTR_DISP,ATCR_SCORE,rank
series_name,,,,,
신상 출시 편스토랑,61,61,61,61,61


In [53]:
drop_data[drop_data['ATCR_SCORE'] == 0.17]

,subsr,asset_nm,ACTR_DISP,series_name,ATCR_SCORE,rank
13,67055000,실화탐사대 230회(23/08/24),"신동엽,서인,강다솜,박지훈",실화탐사대,0.17,180.0
14,67055000,실화탐사대 230회(23/08/24),"신동엽,서인,강다솜,박지훈",실화탐사대,0.17,180.0
15,64376000,실화탐사대 230회(23/08/24),"신동엽,서인,강다솜,박지훈",실화탐사대,0.17,180.0
16,64376000,실화탐사대 230회(23/08/24),"신동엽,서인,강다솜,박지훈",실화탐사대,0.17,180.0
111,65941000,실화탐사대 227회(23/07/27),"신동엽,서인,강다솜,박지훈",실화탐사대,0.17,180.0
...,...,...,...,...,...,...
4724,61931000,(더빙)슈퍼 마리오 브라더스,"크리스 프랫,안야 테일러 조이,잭 블랙,세스 로건,찰리 데이,키건 마이클 키",슈퍼 마리오 브라더스,0.17,180.0
4725,61931000,슈퍼 마리오 브라더스(예고),"크리스 프랫,안야 테일러 조이,잭 블랙,세스 로건,찰리 데이,키건 마이클 키",슈퍼 마리오 브라더스,0.17,180.0
4726,65514000,슈퍼 마리오 브라더스(예고),"크리스 프랫,안야 테일러 조이,잭 블랙,세스 로건,찰리 데이,키건 마이클 키",슈퍼 마리오 브라더스,0.17,180.0
4727,63086000,슈퍼 마리오 브라더스,"크리스 프랫,안야 테일러 조이,잭 블랙,세스 로건,찰리 데이,키건 마이클 키",슈퍼 마리오 브라더스,0.17,180.0


In [54]:
drop_data[drop_data['ATCR_SCORE'] == 0.17].groupby('series_name').count()

,subsr,asset_nm,ACTR_DISP,ATCR_SCORE,rank
series_name,,,,,
나를 잊지 말아요,5,5,5,5,5
미운우리새끼,48,48,48,48,48
슈퍼 마리오 브라더스,6,6,6,6,6
실화탐사대,19,19,19,19,19
익스토션,1,1,1,1,1


In [33]:
(origin_data.groupby(['subsr']).count())[(origin_data.groupby(['subsr']).count())['asset_nm'] == 373]

,Unnamed: 0,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
subsr,,,,,,,,,
64385000,373,373,373,373,373,373,373,373,373


In [49]:
print(view_data.groupby('series_name').count())

                subsr  asset_nm  ACTR_DISP
series_name                               
TV동물농장             94        94         94
걸어서 환장 속으로          3         3          3
고기서 만나              4         4          4
골 때리는 그녀들           8         8          8
과학수사대 스모킹 건         5         5          5
꼬리에꼬리를무는그날이야기       2         2          2
나 혼자산다              2         2          2
내일은 위닝샷             8         8          8
닭, 싸움               3         3          3
동물극장 단짝             3         3          3
런닝맨                 5         5          5
손대면 핫플-동네멋집        16        16         16
식객 허영만의 백반기행        5         5          5
신랑수업               32        32         32
신상 출시 편스토랑         61        61         61
아는 형님               4         4          4
여행설계자들              1         1          1
와이낫크루 - 뜻밖의 여행      6         6          6
웃는 사장               3         3          3
일타강사                3         3          3
전지적 참견 시점          32        32         32
조선의 사랑꾼    

In [108]:
# ACTR score 를 기반으로 상위 컨텐츠 출력

# 불필요한 컬럼 제거
ac_data = drop_data.drop(['subsr', 'asset_nm', 'ACTR_DISP'], axis = 1)
#print(ac_data.head())

# series_name 과 ATCR_SCORE 컬럼을 기준으로 묶어서 중복 제거
count_data = ac_data.groupby(['series_name', 'ATCR_SCORE']).count().reset_index()
#print(count_data.head())

# ATCR_SCORE 를 기준으로 정렬을 수행
sorted_ac_data = count_data.sort_values(by = ['ATCR_SCORE', 'series_name'], ascending = False)
#print(type(sorted_ac_data))
#print(sorted_ac_data.head())

# 최종 정렬 결과 확인
print(sorted_ac_data)

      series_name  ATCR_SCORE  rank
13         TV동물농장        0.68   118
394    신상 출시 편스토랑        0.33    61
527          익스토션        0.17     1
399         실화탐사대        0.17    19
372   슈퍼 마리오 브라더스        0.17     6
..            ...         ...   ...
4    2023 역사저널 그날        0.00     4
3    2022 역사저널 그날        0.00     3
2        1박2일 시즌4        0.00    25
1       19회 고창 1부        0.00     1
0        10일간의 애인        0.00     5

[751 rows x 3 columns]


In [111]:
# 상위 5개 컨텐츠의 정보를 리스트로 생성
top5 = list(sorted_ac_data['series_name'][:5])
print(top5)

['TV동물농장', '신상 출시 편스토랑', '익스토션', '실화탐사대', '슈퍼 마리오 브라더스']
